<a href="https://colab.research.google.com/github/OmDuggineni/GPT2Experiments/blob/master/translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Dataset is from  https://www.kaggle.com/kkhandekar/hindi-english-sentence-pairs

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install indic-transliteration
from indic_transliteration import sanscript 
from indic_transliteration.sanscript import transliterate 

In [ ]:
import pandas as pd;
sentences_df = pd.read_csv("hin.tsv",sep="\t");
del sentences_df["Attribution"];
sentences_df = sentences_df.sample(frac=1).reset_index(drop=True);
sentences_df.head()

,English,Hindi
0,There is a bag on the desk.,मेज़ पर बस्ता रखा हुआ है।
1,She had a daughter by her first husband.,उसकी अपने पहले पति से एक लड़की थी।
2,What is this?,यह क्या है?
3,I heard someone call my name from behind.,मैंने किसी को पीछे से मेरे नाम पुकारते हुए सुना।
4,I'm an atheist.,मैं भगवान में यकीन नहीं करता।


In [ ]:
for i in sentences_df.index: 
    sentences_df["Hindi"][i] = transliterate(sentences_df["Hindi"][i], sanscript.DEVANAGARI, sanscript.ITRANS);
sentences_df.head()

,English,Hindi
0,There is a bag on the desk.,meja़ para bastA rakhA huA hai|
1,She had a daughter by her first husband.,usakI apane pahale pati se eka la.DakI thI|
2,What is this?,yaha kyA hai?
3,I heard someone call my name from behind.,maiMne kisI ko pIChe se mere nAma pukArate hue...
4,I'm an atheist.,maiM bhagavAna meM yakIna nahIM karatA|


In [ ]:
sentences_df_prefix = sentences_df.head(5);

In [ ]:
#comment this line out and uncomment the one further down that reads the pefix if you want the model to not use a prefix
#sentences_df = sentences_df.tail(-5);

In [ ]:
sentences_df.to_csv("translation.txt",sep = "\t", index=False, header=False);
sentences_df_prefix.to_csv("prefix.txt",sep = "\t", index=False, header=False);

In [ ]:
!cat translation.txt

There is a bag on the desk.	meja़ para bastA rakhA huA hai|
She had a daughter by her first husband.	usakI apane pahale pati se eka la.DakI thI|
What is this?	yaha kyA hai?
I heard someone call my name from behind.	maiMne kisI ko pIChe se mere nAma pukArate hue sunA|
I'm an atheist.	maiM bhagavAna meM yakIna nahIM karatA|
He has no sense of right and wrong.	vaha sahI-ga़lata meM aMtara karanA nahIM jAnatA|
Something is wrong with this washing machine.	isa vaॉshiMga mashIna meM kuCha kharAbI hai|
I want this photograph developed as soon as possible.	mujhe yaha tasvIra jalda-se-jalda banavAnI hai|
You reap what you sow.	kara bhalA to ho bhalA|
We were surprised at the news.	hama khabara sunakara chauMka gae|
I've lost my glasses.	mere chashme kho gae haiM|
How are you?	Apa kaise haiM?
I believe whatever he says.	vaha jo bhI kahe, mujhe usa para yakIna hai|
It would be better for you not to ask him for advice.	bahatara hogA ki tuma usase salAha na mA.Ngo|
All the students look up to him.	

In [ ]:
#Comment out if not using prefix
#!cat prefix.txt

In [11]:
%tensorflow_version 1.x
#!pip install --upgrade pip
!pip install toposort
!pip install gpt-2-simple

TensorFlow 1.x selected.
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.1-cp36-none-any.whl size=23581 sha256=6d7827bda8c2cde06beb884906cd23478b440ca4396126330d239f298dbf2b16
  Stored in directory: /root/.cache/pip/wheels/0c/f8/23/b53ce437504597edff76bf9c3b8de08ad716f74f6c6baaa91a
Successfully built gpt-2-simple


In [12]:
import gpt_2_simple as gpt2;
model_name="355M"
from tensorflow.python.client import device_lib
if "P100" in str(device_lib.list_local_devices()):
  print("Detected NVIDIA Tesla P100 GPU. Using larger model size.");
  model_name="774M";
else:
  print("Did not detect P100 GPU.")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Did not detect P100 GPU.


In [13]:
gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 351Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 60.6Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 362Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:22, 63.2Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 149Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 59.1Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 119Mit/s]                                                       


In [14]:
sess = gpt2.start_tf_sess()

In [ ]:
gpt2.finetune(sess,
              dataset="translation.txt",
              model_name=model_name,
              steps=500,
              restore_from='fresh',
              run_name='run1')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.91it/s]


dataset has 81047 tokens
Training...
[1 | 13.79] loss=4.38 avg=4.38
[2 | 17.00] loss=4.66 avg=4.52
[3 | 20.23] loss=4.15 avg=4.40
[4 | 23.45] loss=4.12 avg=4.33
[5 | 26.67] loss=4.04 avg=4.27
[6 | 29.89] loss=3.94 avg=4.21
[7 | 33.10] loss=3.80 avg=4.15
[8 | 36.31] loss=3.68 avg=4.09
[9 | 39.51] loss=3.66 avg=4.04
[10 | 42.73] loss=3.78 avg=4.01
[11 | 45.93] loss=3.29 avg=3.94
[12 | 49.14] loss=3.65 avg=3.92
[13 | 52.34] loss=3.35 avg=3.87
[14 | 55.56] loss=3.54 avg=3.85
[15 | 58.79] loss=3.32 avg=3.81
[16 | 62.02] loss=3.32 avg=3.78
[17 | 65.24] loss=3.11 avg=3.73
[18 | 68.46] loss=3.18 avg=3.70
[19 | 71.69] loss=3.40 avg=3.68
[20 | 74.89] loss=3.14 avg=3.65
[21 | 78.12] loss=3.15 avg=3.63
[22 | 81.32] loss=3.25 avg=3.61
[23 | 84.54] loss=3.17 avg=3.59
[24 | 87.74] loss=2.71 avg=3.55
[25 | 90.96] loss=2.69 avg=3.51
[26 | 94.16] loss=3.08 avg=3.49
[27 | 97.37] loss=3.03 avg=3.47
[28 | 100.59] loss=2.92 avg=3.45
[29 | 103.80] loss=2.90 avg=3.43
[30 | 107.02] loss=2.99 avg=3.41
[31 | 110

In [ ]:
#Credit for this goes to https://stackoverflow.com/a/31505798 (Answer licensed under CC BY-SA 4.0)

import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  ";
    text = text.replace("\n"," ");
    text = re.sub(prefixes,"\\1<prd>",text);
    text = re.sub(websites,"<prd>\\1",text);
    
    #if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>");
    text = text.replace("Ph.D.","Ph<prd>D<prd>");
    
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text);
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text);
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text);
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text);
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text);
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text);
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text);
    
    #if "”" in text: text = text.replace(".”","”.");
    text = text.replace(".”","”.");

    #if "\"" in text: text = text.replace(".\"","\".");
    text = text.replace(".\"","\".");

    #if "!" in text: text = text.replace("!\"","\"!");
    text = text.replace("!\"","\"!");

    #if "?" in text: text = text.replace("?\"","\"?");
    text = text.replace("?\"","\"?");
    
    text = text.replace(".",".<stop>");
    text = text.replace("?","?<stop>");
    text = text.replace("!","!<stop>");
    text = text.replace("<prd>",".");
    sentences = text.split("<stop>");
    sentences = sentences[:-1];
    sentences = [s.strip() for s in sentences];
    return sentences;

In [ ]:
#prefix = open('prefix.txt', 'r').read().rstrip('\n');
def translate_sentence(input):
    global prefix;
    #comment this line and uncomment the one further upward if you want the model to not use a prefix
    #full_prefix = prefix + "\n" + input + "\t";
    full_prefix = input + "\t";
    output = gpt2.generate(sess, model_name=model_name, prefix=full_prefix, length=30, temperature=0.1, top_p=0.9, return_as_list=True)[0];
    #return output.replace(full_prefix, "").split("\n")[0].strip();
    return transliterate(output.replace(full_prefix, "").split("\n")[0].strip(), sanscript.ITRANS, sanscript.DEVANAGARI);

In [ ]:
def translate(input):
  if len(input.split()) == 1:
    return translate_sentence(input);
  sentences = split_into_sentences(input);
  #print(sentences)
  sentences = [translate_sentence(sentence) for sentence in sentences];
  #print(sentences);
  return "\n".join(sentences);

In [ ]:
print(translate("Hello! My name is Om."));

मैं!
मैं बात नहीं का सकता।


In [ ]:
print(translate("How are you?"));

आप कैसे हो?


In [ ]:
print(translate("Jump."));

मुझे लगता है।


In [ ]:
print(translate("Yes"));

मुझे यह सब नगरठी।


In [ ]:
print(translate("What?"));

मुझे अभी भी सारा काम क्या?


In [ ]:
#print(translate(input()));